In [1]:
import os
from collections import OrderedDict
import torch
import torch.nn.functional as F
from src.util import (
    make_faces,    
)    

def vertex_tris(faces):
    res = [[] for _ in range(faces.max()+1)]
    for fid, face in enumerate(faces):        
        for vid in face:
            res[vid].append(fid)        
    return res

def vertex_tri_maps(faces):
    vts = vertex_tris(faces)
    r, c = len(vts), max([len(x) for  x in vts])
    vert_tri_indices = torch.zeros(r, c, dtype=torch.long)
    vert_tri_weights = torch.zeros(r, c)    
    for r, tris in enumerate(vts):        
        weight = 1. #/ len(tris)
        for c, tri_id in enumerate(tris):
            vert_tri_indices[r, c] = tri_id
            vert_tri_weights[r, c] = weight
    return vert_tri_indices, vert_tri_weights.unsqueeze(dim=-1)[None]


In [2]:
faces = make_faces(3, 3)
faces

array([[3, 0, 4],
       [1, 4, 0],
       [4, 1, 5],
       [2, 5, 1],
       [6, 3, 7],
       [4, 7, 3],
       [7, 4, 8],
       [5, 8, 4]])

In [3]:
vertex_tris(faces)

[[0, 1],
 [1, 2, 3],
 [3],
 [0, 4, 5],
 [0, 1, 2, 5, 6, 7],
 [2, 3, 7],
 [4],
 [4, 5, 6],
 [6, 7]]

In [4]:
# bs vrt_no 3 3
vrt = torch.rand(1, 9, 3)
vrt.shape

torch.Size([1, 9, 3])

In [5]:
vrt_no = vrt.shape[1]
vrt_no

9

In [19]:
angle_sel = torch.randint(0, vrt_no, (vrt_no, 6, 3))
print(angle_sel.shape)
angle_sel

torch.Size([9, 6, 3])


tensor([[[1, 3, 5],
         [0, 2, 4],
         [4, 7, 2],
         [4, 8, 8],
         [4, 5, 6],
         [6, 4, 0]],

        [[7, 4, 1],
         [5, 6, 7],
         [2, 3, 6],
         [6, 3, 3],
         [7, 0, 3],
         [4, 3, 7]],

        [[4, 7, 7],
         [0, 6, 0],
         [3, 6, 1],
         [2, 5, 1],
         [5, 4, 7],
         [1, 3, 8]],

        [[8, 6, 5],
         [7, 8, 8],
         [5, 2, 7],
         [8, 0, 6],
         [6, 8, 8],
         [7, 6, 6]],

        [[6, 5, 3],
         [1, 5, 0],
         [7, 0, 8],
         [4, 4, 2],
         [7, 0, 8],
         [5, 3, 5]],

        [[4, 2, 0],
         [5, 1, 1],
         [0, 5, 6],
         [6, 2, 3],
         [7, 8, 5],
         [2, 3, 0]],

        [[3, 6, 2],
         [8, 8, 0],
         [0, 7, 7],
         [1, 7, 5],
         [6, 3, 8],
         [3, 0, 5]],

        [[0, 3, 0],
         [1, 4, 0],
         [3, 4, 6],
         [7, 2, 6],
         [4, 3, 6],
         [7, 3, 2]],

        [[5, 5, 2],
    

In [21]:
vrt.shape
angle_pts = vrt.index_select(1, angle_sel.view(-1)).reshape(1, 9, 6, 3, 3)
print(angle_pts.shape)
angle_pts

torch.Size([1, 9, 6, 3, 3])


tensor([[[[[0.3252, 0.9688, 0.8584],
           [0.1242, 0.2599, 0.5692],
           [0.5787, 0.5036, 0.5874]],

          [[0.2990, 0.7059, 0.3558],
           [0.4478, 0.2418, 0.3900],
           [0.3643, 0.3132, 0.3545]],

          [[0.3643, 0.3132, 0.3545],
           [0.1571, 0.1381, 0.4521],
           [0.4478, 0.2418, 0.3900]],

          [[0.3643, 0.3132, 0.3545],
           [0.6385, 0.9155, 0.9512],
           [0.6385, 0.9155, 0.9512]],

          [[0.3643, 0.3132, 0.3545],
           [0.5787, 0.5036, 0.5874],
           [0.8140, 0.4826, 0.2013]],

          [[0.8140, 0.4826, 0.2013],
           [0.3643, 0.3132, 0.3545],
           [0.2990, 0.7059, 0.3558]]],


         [[[0.1571, 0.1381, 0.4521],
           [0.3643, 0.3132, 0.3545],
           [0.3252, 0.9688, 0.8584]],

          [[0.5787, 0.5036, 0.5874],
           [0.8140, 0.4826, 0.2013],
           [0.1571, 0.1381, 0.4521]],

          [[0.4478, 0.2418, 0.3900],
           [0.1242, 0.2599, 0.5692],
           [0.8140, 

In [22]:
print(angle_pts.shape)
a = angle_pts[:, :, :, 0]
b = angle_pts[:, :, :, 1]
c = angle_pts[:, :, :, 2]

ba = a - b
bc = c - b

#cosine_angle = torch.dot(ba, bc) / (torch.norm(ba) * torch.norm(bc))
#torch.arccos(cosine_angle)
# torch.arccos(torch.dot(ba / torch.norm(ba), bc / torch.norm(bc)))

torch.Size([1, 9, 6, 3, 3])


In [28]:
ba.shape, torch.norm(ba, dim=-1).unsqueeze(-1).shape

(torch.Size([1, 9, 6, 3]), torch.Size([1, 9, 6, 1]))

In [39]:
baNrm

tensor([[[[ 0.2539,  0.8956,  0.3653],
          [-0.3046,  0.9499, -0.0699],
          [ 0.7187,  0.6074, -0.3384],
          [-0.3076, -0.6760, -0.6696],
          [-0.5803, -0.5155, -0.6305],
          [ 0.8916,  0.3359, -0.3038]],

         [[-0.7187, -0.6074,  0.3384],
          [-0.5197,  0.0465,  0.8531],
          [ 0.8737, -0.0487, -0.4840],
          [ 0.8485,  0.2740, -0.4527],
          [-0.2392, -0.9573,  0.1622],
          [ 0.7354,  0.1632, -0.6577]],

         [[ 0.7187,  0.6074, -0.3384],
          [-0.8845,  0.3835,  0.2655],
          [-0.8485, -0.2740,  0.4527],
          [-0.3707, -0.7415, -0.5592],
          [ 0.5803,  0.5155,  0.6305],
          [ 0.2539,  0.8956,  0.3653]],

         [[-0.1986,  0.4900,  0.8488],
          [-0.4620, -0.7463, -0.4791],
          [ 0.3707,  0.7415,  0.5592],
          [ 0.4736,  0.2926,  0.8307],
          [ 0.1986, -0.4900, -0.8488],
          [-0.8389, -0.4400,  0.3203]],

         [[ 0.5197, -0.0465, -0.8531],
          [-0.426

In [31]:
baNrm = ba / torch.norm(ba, dim=-1).unsqueeze(-1)
bcNrm = bc / torch.norm(bc, dim=-1).unsqueeze(-1)
baNrm.shape, bcNrm.shape

(torch.Size([1, 9, 6, 3]), torch.Size([1, 9, 6, 3]))

In [36]:
dotBac = (baNrm * bcNrm).sum(dim=-1).unsqueeze(-1)
dotBac.shape

torch.Size([1, 9, 6, 1])

In [38]:
torch.arccos(dotBac).shape

torch.Size([1, 9, 6, 1])